In [3]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    [
        "Harrison works at Langchain",
        "Karen works at Snowflake",
        "Gor works at Microsoft",
        "Habet works at AUA",
        "Bears like to eat honey",
    ],
    embedding=OpenAIEmbeddings(),
)


In [4]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever.invoke("Where does Karen work?")

[Document(page_content='Karen works at Snowflake'),
 Document(page_content='Gor works at Microsoft')]

In [24]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
)
chain = setup_and_retrieval | prompt | model | output_parser

In [25]:
chain.invoke("what bears like to eat?")

'Bears like to eat honey.'

<img src="image.svg"/>